## Exemplo de Acesso à API do Twinscie

Este notebook descreve o processo de acesso à API do Twinscie utilizando a linguagem `Python`, por meio da biblioteca [requests](https://requests.readthedocs.io/en/latest/).

A sequência de operações descritas abaixo possui como objetivo demonstrar o processo 
de inferência do NeMo através da API, o qual envolve o cadastro de metadados (domínio, projeto, learner_family, execution_environment), bem como o upload de datasets, dataset_processors e models.

A documentação completa da API, gerada pelo framework Swagger, pode ser consultada em http://localhost:8080/api/swagger-ui.

A imagem abaixo ilustra um exemplo da sequência de chamadas à API visando a execução de um processo de predição:

![Example Image](api_prediction_flow.jpg)<!---fig:figure1--->

### Importação das bibliotecas necessárias

As requisições HTTP à API serão realizadas por meio da biblioteca [requests](https://requests.readthedocs.io/en/latest/). Dessa forma, o primeiro passo é a importação desta biblioteca. Além disso, é necessário importar a biblioteca getpass para anonimização do password de usuário utilizado na autenticação (quando cabível).

In [ ]:
import requests
import getpass

### Definição da URL e da informação de autenticação

Para realizar as requisições, é necessário antes definir a URL onde a API encontra-se hospedada.
Além disso, para casos onde as requisições são realizadas para o ambiente de homologação e testes 
hospedado no LNCC (https://intelipetro-gypscie.lncc.br/), é necessário definir usuário e senha para autenticação.
Caso você não possua credenciais de acesso, basta solicitá-las à equipe de desenvolvimento do Twinscie.

In [ ]:
# Ambiente local
URL = "http://localhost:8080/api/"

In [ ]:
# Ambiente de homologação no LNCC
ULR = "https://intelipetro-gypscie.lncc.br/api/"

In [ ]:
USER = input("Usuário: ")
PASSWORD = getpass.getpass("Senha: ")

## Preparação do Ambiente

Como demonstrado na figura abaixo, é necessário cadastrar um conjunto de metadados gerenciados 
pelo Twinscie (Domain, Project e ExecutionEnvironment) e que serão necessários em etapas posteriores do processo de predição.

![Example Image](environment_prep.jpg)

### Cadastro de *Domain*

In [ ]:
# Requisição de cadastro de Domain
response = requests.post(
    url=URL + "domains",
    json={
        "name": "domain 1",
        "description": "domain 1 description"
    },
    auth=(USER, PASSWORD)
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

### Cadastro de *Project*

In [ ]:
# Requisição de cadastro de Project
response = requests.post(
    url=URL + "projects",
    json={
    "name": "project_1",
    "description": "project 1 description",
    "domain_id": 1
    }
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

### Cadastro de *ExecutionEnvironment*

In [ ]:
# Requisição de cadastro de ExecutionEnvironment (ambiente local)
response = requests.post(
    url = URL + "execution_environments",
    json = {
        "name": "environment_1",
        "uri": "localhost",
        "arguments": {},
        "port": 50052,
        
    },
)

In [ ]:
# Requisição de cadastro de ExecutionEnvironment (ambiente de homologação no LNCC)
response = requests.post(
    url = URL + "execution_environments",
    json = {
        "name": "environment_1",
        "uri": "modelserver",
        "arguments": {},
        "port": 50052,
        
    },
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

## Pré-processamento dos dados

Outro funcionalidade disponibilizada pelo Twinscie é a possibilidade de pré-processamento dos dados de modo a 
prepará-los para o processo de predição. Para tanto, é necessário que o(s) dataset(s) utilizados no processo sejam 
cadastrados, bem como o artefato denominado DatasetProcessor. A figura abaixo ilustra esse processo.

![Example Figure](dataset_processor.jpg)

### Cadastro de *Datasets*

##### Adicionando arquivo monk.pos

In [ ]:
# Requisição de cadastro e upload de um Dataset
response = requests.post(
    url = URL + "datasets",
    data = {
        "name": "nemo_pos",
        "domain_id": 1,
    },
    files = {
        "files": open(file="monk.pos", mode="rb"),
    }
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

##### Adicionando arquivo monk.mot

In [ ]:
# Requisição de cadastro e upload de um Dataset
response = requests.post(
    url = URL + "datasets",
    data = {
        "name": "nemo_mot",
        "domain_id": 1,
    },
    files = {
        "files": open(file="monk.mot", mode="rb"),
    }
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

### Cadastro de *DatasetProcessor*

In [ ]:
# Requisição de cadastro e upload de um DatasetProcessor
response = requests.post(
    url = URL + "dataset_processors",
    data = {
        "name": "nemo_processor",
        "description":  "nemo_processor description",
        "input_arity":  "many",
        "output_arity": "one",
        "processor_type": "transformer"
    },
    files = {
        "files": open(file="nemo_dataset_processor.zip", mode="rb"),
    }
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

###  Execução do *DatasetProcessor*

Todos os processos de execução no Twinscie são caracterizados por serem assíncronos. O resultado prático desta 
característica é que as requisições para execução retornam uma `task_id`, a qual pode ser utilizada para verificar o 
status de execução da tarefa, bem como os ids dos datasets gerados ao final da execução (caso seja bem sucedida).

In [ ]:
# Requisição de execução de um DatasetProcessor
response = requests.post(
    url = URL + "processor_run",
    json = {
        "processor_id": 1,
        "dataset_id": [1,2],
        "environment_id": 1,
        "parameters": {},
        "project_id": 1
    }
)

In [ ]:
# Verificando o status_code retornado pela API
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

In [ ]:
task_id = response.json().get('task_id')

In [ ]:
task_id

### Monitorando status de execução do *DatasetProcessor*

In [ ]:
# Requisição para verificação do status de execução de um DatasetProcessor
response = requests.get(
    url = URL + "status_processor_run/" + task_id,
)

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

## Execução da Predição

O último conjunto de operações, apresentado na figura abaixo, envolve o cadastro de uma *LearnerFamily*, a importação do modelo serializado do NeMo e a execução deste modelo, utilizando como entrada o *dataset* gerado pela execução do 
*DatasetProcessor*.

![Figure](prediction.jpg)

### Cadastro de uma *LearnerFamily*

In [ ]:
# Requisição para o cadastro de uma LearnerFamily
response = requests.post(
    url = URL + "learner_families",
    json = {
        "name": "learner_family_1",
        "description": "learner_family_1 description"
    }
)

In [ ]:
# Verificando a resposta retornada à requisição
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

### Importação do *Model*

In [ ]:
# Requisição para o cadastro e importação do modelo
response = requests.post(
    url = URL + "models",
    data = {
        "name": "nemo",
        "description":  "nemo description",
        "supervision_scope":  "supervised",
        "task_type": "regression",
        "tool": "keras",
        "learner_family_id": 1
    },
    files = {
    "files": open(file="model.h5", mode="rb"), # O model.h5 não está sendo versionado devido ao seu tamanho (~700MB)
    },
)

In [ ]:
# Verificando a resposta retornada à requisição
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

### Execução da Predição

Por se tratar de uma execução, o processo de predição retorna uma `task_id`, a qual pode ser utilizada para verificar o status de execução da tarefa, bem como os ids dos datasets gerados ao final da execução (caso seja bem sucedida).

In [ ]:
# Requisição de execução de um processo de Predição
response = requests.post(
    url = URL + "predict",
    json = {
        "model_id": 1,
        "dataset_id": 3,
        "parameters": {},
        "project_id": 1
    }
)

In [ ]:
# Verificando a resposta retornada à requisição
response.status_code

In [ ]:
# Verificando a resposta retornada à requisição
response.json()

In [ ]:
task_id = response.json().get('task_id')

In [ ]:
task_id

### Monitorando status de execução da Predição

In [ ]:
response = requests.get(
    url = URL + "status_predict/" + task_id,
)

In [ ]:
response.json()

### Verificando *Dataset* de saída

In [ ]:
response = requests.get(
    url = URL + "datasets/" + "<dataset_id>"
)

In [ ]:
response.json()